# CLIENTE RAG

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "***REMOVED***"

import openai
client = openai.OpenAI()

In [7]:
import inspect
import openai
from functools import lru_cache

_cache = {}

def make_cache_key(kwargs):
    def convert_to_hashable(value):
        if isinstance(value, dict):
            return frozenset((k, convert_to_hashable(v)) for k, v in sorted(value.items()))
        elif isinstance(value, list):
            return tuple(convert_to_hashable(v) for v in value)
        elif isinstance(value, set):
            return frozenset(convert_to_hashable(v) for v in value)
        else:
            return value

    return frozenset((k, convert_to_hashable(v)) for k, v in sorted(kwargs.items()))

def llm(**kwargs):
    
    key = make_cache_key(kwargs)
    try:
        return _cache[key]
    except KeyError:
        _cache[key] = result = client.chat.completions.create(**kwargs)
        return result

@lru_cache(3000)
def get_embedding(text, model="text-embedding-3-large"):

    client = openai.OpenAI()
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding


def get_module_functions(module_name):
    module = __import__(module_name)
    functions = []

    for name, obj in inspect.getmembers(module):
        if inspect.isfunction(obj) and obj.__module__ == module_name:
            ds = inspect.getdoc(obj)
            function_info = {
                "name": name,
                "docstring": ds
            }
            if ds:
                functions.append(function_info)

    return functions

In [39]:
import inspect
import builtins

def get_module_builtins(module_name):
    # Importar el módulo usando __import__
    module = __import__(module_name)
    functions = []

    # Obtener todos los miembros del módulo
    for name, obj in inspect.getmembers(module):
        # Verificar si el objeto es una función
        if inspect.isfunction(obj) or inspect.isbuiltin(obj):
            # Obtener la docstring de la función
            ds = inspect.getdoc(obj)
            function_info = {
                "name": name,
                "docstring": ds
            }
            # Añadir la función a la lista si tiene una docstring
            if ds:
                functions.append(function_info)

    return functions


In [19]:
# Ejemplo de uso
builtins_functions = get_module_builtins("builtins")
print(len(builtins_functions))
for func in builtins_functions:
    # print(func["name"], ":", func["docstring"])
    print(func["name"])


25

In [41]:
names = [x["name"] for x in builtins_functions]
chunks = [x["docstring"] for x in builtins_functions]
embeddings = [get_embedding(name + "\n\n" + doc[:50]) for (name, doc) in zip(names, chunks)]

In [43]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def query_bbdd(query, top_n=5):
    query_embedding = get_embedding(query)
    
    # Calcula la similitud coseno entre la consulta y cada embedding de los chunks
    similarities = [cosine_similarity(query_embedding, embedding) for embedding in embeddings]
    
    # Obtiene los índices de los top-n chunks más similares
    top_indices = np.argsort(similarities)[-top_n:]
    
    # Recupera los top-n chunks más similares junto con su similitud coseno
    top_chunks = [(names[i], chunks[i], similarities[i]) for i in top_indices]
    
    return top_chunks